Script para hacer extraccion y tagging asi como analisis de texto, sentimientos. etc. con el framework de Langchain

In [34]:
import os
import openai
import bs4
from typing import Optional
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from typing import List
from pydantic import BaseModel, Field

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['API_GPT_KEY']

ModuleNotFoundError: No module named 'kor'

In [32]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [33]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [14]:
doc = documents[0]

In [15]:
page_content = doc.page_content[:10000]

Hasta aquí se extrajo perfectamente las primeras 10.000 caracteres.

In [16]:
print(page_content[:1000])







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
June 23, 2023 · 31 min · Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [20]:
splits = text_splitter.split_text(doc.page_content)

In [21]:
len(splits)

14

In [22]:
def flatten(matrix):
    #function for join lists of lists 
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [23]:
flatten([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [24]:
from langchain.schema.runnable import RunnableLambda

In [25]:
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [26]:
prep.invoke("hi")

[{'input': 'hi'}]

In [ ]:
chain = prep | extraction_chain.map() | flatten

In [ ]:
chain.invoke(doc.page_content)

In [17]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [ ]:
overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [ ]:
tagging_chain.invoke({"input": page_content})

In [ ]:
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]